# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 12 2022 8:56AM,252785,10,0086237783,ISDIN Corporation,Released
1,Dec 12 2022 8:56AM,252785,10,0086237784,ISDIN Corporation,Released
2,Dec 12 2022 8:56AM,252785,10,0086237786,ISDIN Corporation,Released
3,Dec 12 2022 8:56AM,252785,10,0086237804,ISDIN Corporation,Released
4,Dec 12 2022 8:56AM,252785,10,0086237806,ISDIN Corporation,Released
5,Dec 12 2022 8:56AM,252785,10,0086237881,ISDIN Corporation,Released
6,Dec 12 2022 8:56AM,252785,10,0086239119,ISDIN Corporation,Released
7,Dec 12 2022 8:56AM,252785,10,0086241945,ISDIN Corporation,Released
8,Dec 12 2022 8:55AM,252786,20,8892009,"Exact-Rx, Inc.",Released
9,Dec 12 2022 8:54AM,252784,19,TR-RN-12122022-PK1,"GCH Granules USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,252782,Released,19
8,252783,Released,1
9,252784,Released,1
10,252785,Released,8
11,252786,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
252782,NaN,19.0
252783,NaN,1.0
252784,NaN,1.0
252785,NaN,8.0
252786,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
252771,0.0,7.0
252772,6.0,7.0
252777,0.0,1.0
252778,0.0,1.0
252779,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
252771,0,7
252772,6,7
252777,0,1
252778,0,1
252779,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,252771,0,7
1,252772,6,7
2,252777,0,1
3,252778,0,1
4,252779,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,252771,,7
1,252772,6,7
2,252777,,1
3,252778,,1
4,252779,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 12 2022 8:56AM,252785,10,ISDIN Corporation
8,Dec 12 2022 8:55AM,252786,20,"Exact-Rx, Inc."
9,Dec 12 2022 8:54AM,252784,19,"GCH Granules USA, Inc."
10,Dec 12 2022 8:52AM,252783,12,Hush Hush
11,Dec 12 2022 8:51AM,252782,12,Hush Hush
30,Dec 12 2022 8:45AM,252780,22,"NBTY Global, Inc."
31,Dec 12 2022 8:45AM,252778,20,"HVL, LLC dba Atrium Innovations"
32,Dec 12 2022 8:45AM,252779,22,"NBTY Global, Inc."
33,Dec 12 2022 8:32AM,252777,20,Else Nutrition
34,Dec 12 2022 7:42AM,252772,19,"Innogenix, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 12 2022 8:56AM,252785,10,ISDIN Corporation,,8
1,Dec 12 2022 8:55AM,252786,20,"Exact-Rx, Inc.",,1
2,Dec 12 2022 8:54AM,252784,19,"GCH Granules USA, Inc.",,1
3,Dec 12 2022 8:52AM,252783,12,Hush Hush,,1
4,Dec 12 2022 8:51AM,252782,12,Hush Hush,,19
5,Dec 12 2022 8:45AM,252780,22,"NBTY Global, Inc.",,1
6,Dec 12 2022 8:45AM,252778,20,"HVL, LLC dba Atrium Innovations",,1
7,Dec 12 2022 8:45AM,252779,22,"NBTY Global, Inc.",,1
8,Dec 12 2022 8:32AM,252777,20,Else Nutrition,,1
9,Dec 12 2022 7:42AM,252772,19,"Innogenix, LLC",6,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 8:56AM,252785,10,ISDIN Corporation,8,
1,Dec 12 2022 8:55AM,252786,20,"Exact-Rx, Inc.",1,
2,Dec 12 2022 8:54AM,252784,19,"GCH Granules USA, Inc.",1,
3,Dec 12 2022 8:52AM,252783,12,Hush Hush,1,
4,Dec 12 2022 8:51AM,252782,12,Hush Hush,19,
5,Dec 12 2022 8:45AM,252780,22,"NBTY Global, Inc.",1,
6,Dec 12 2022 8:45AM,252778,20,"HVL, LLC dba Atrium Innovations",1,
7,Dec 12 2022 8:45AM,252779,22,"NBTY Global, Inc.",1,
8,Dec 12 2022 8:32AM,252777,20,Else Nutrition,1,
9,Dec 12 2022 7:42AM,252772,19,"Innogenix, LLC",7,6


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 8:56AM,252785,10,ISDIN Corporation,8,
1,Dec 12 2022 8:55AM,252786,20,"Exact-Rx, Inc.",1,
2,Dec 12 2022 8:54AM,252784,19,"GCH Granules USA, Inc.",1,
3,Dec 12 2022 8:52AM,252783,12,Hush Hush,1,
4,Dec 12 2022 8:51AM,252782,12,Hush Hush,19,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 8:56AM,252785,10,ISDIN Corporation,8,NaN
1,Dec 12 2022 8:55AM,252786,20,"Exact-Rx, Inc.",1,NaN
2,Dec 12 2022 8:54AM,252784,19,"GCH Granules USA, Inc.",1,NaN
3,Dec 12 2022 8:52AM,252783,12,Hush Hush,1,NaN
4,Dec 12 2022 8:51AM,252782,12,Hush Hush,19,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 12 2022 8:56AM,252785,10,ISDIN Corporation,8,0.0
1,Dec 12 2022 8:55AM,252786,20,"Exact-Rx, Inc.",1,0.0
2,Dec 12 2022 8:54AM,252784,19,"GCH Granules USA, Inc.",1,0.0
3,Dec 12 2022 8:52AM,252783,12,Hush Hush,1,0.0
4,Dec 12 2022 8:51AM,252782,12,Hush Hush,19,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,252785,8,0.0
12,505565,20,0.0
19,758327,15,6.0
20,758341,3,0.0
22,505559,2,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,252785,8,0.0
1,12,505565,20,0.0
2,19,758327,15,6.0
3,20,758341,3,0.0
4,22,505559,2,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,8,0.0
1,12,20,0.0
2,19,15,6.0
3,20,3,0.0
4,22,2,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,12,Released,20.0
2,19,Released,15.0
3,20,Released,3.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,20,22
Status,,,,,
Executing,0.0,0.0,6.0,0.0,0.0
Released,8.0,20.0,15.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,20,22
0,Executing,0.0,0.0,6.0,0.0,0.0
1,Released,8.0,20.0,15.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,20,22
0,Executing,0.0,0.0,6.0,0.0,0.0
1,Released,8.0,20.0,15.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()